# Student Alcohol Consumption

### Introduction:

This time you will download a dataset from the UCI.

### Step 1. Import the necessary libraries

In [8]:
import pandas as pd
import numpy as np
from pyspark.sql import SparkSession
import requests

### Step 2. Import the dataset from this [address](https://raw.githubusercontent.com/guipsamora/pandas_exercises/master/04_Apply/Students_Alcohol_Consumption/student-mat.csv).

### Step 3. Assign it to a variable called df.

In [9]:

url = 'https://raw.githubusercontent.com/guipsamora/pandas_exercises/master/04_Apply/Students_Alcohol_Consumption/student-mat.csv'


get_url = requests.get(url)


with open('data.csv', 'w', encoding='utf-8') as f:
    
    f.write(get_url.text)
    

df = pd.read_csv('data.csv', header=0, sep=',')

spark = SparkSession.builder\
                    .appName('students')\
                    .getOrCreate()
                    


students = spark.read.csv('data.csv', header=True, sep=',')

### Step 4. For the purpose of this exercise slice the dataframe from 'school' until the 'guardian' column

In [10]:

columnas = students.columns

inicio = columnas.index('school')
final = columnas.index('guardian')

subset_columnas = columnas[inicio:final]


students_subset = students.select(subset_columnas)


In [11]:
df_subset = df.loc[:, 'school':'guardian']
df_subset

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,reason,guardian
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,course,mother
1,GP,F,17,U,GT3,T,1,1,at_home,other,course,father
2,GP,F,15,U,LE3,T,1,1,at_home,other,other,mother
3,GP,F,15,U,GT3,T,4,2,health,services,home,mother
4,GP,F,16,U,GT3,T,3,3,other,other,home,father
...,...,...,...,...,...,...,...,...,...,...,...,...
390,MS,M,20,U,LE3,A,2,2,services,services,course,other
391,MS,M,17,U,LE3,T,3,1,services,services,course,mother
392,MS,M,21,R,GT3,T,1,1,other,other,course,other
393,MS,M,18,R,LE3,T,3,2,services,other,course,mother


### Step 5. Create a lambda function that will capitalize strings.

In [12]:
from pyspark.sql import functions as F

capitalize_udf = F.udf(lambda x: str(x).capitalize() if x is not None else None, F.StringType())


students_subset.withColumn("col_capitalizada", capitalize_udf(students_subset["reason"])).show()


+------+---+---+-------+-------+-------+----+----+--------+--------+----------+----------------+
|school|sex|age|address|famsize|Pstatus|Medu|Fedu|    Mjob|    Fjob|    reason|col_capitalizada|
+------+---+---+-------+-------+-------+----+----+--------+--------+----------+----------------+
|    GP|  F| 18|      U|    GT3|      A|   4|   4| at_home| teacher|    course|          Course|
|    GP|  F| 17|      U|    GT3|      T|   1|   1| at_home|   other|    course|          Course|
|    GP|  F| 15|      U|    LE3|      T|   1|   1| at_home|   other|     other|           Other|
|    GP|  F| 15|      U|    GT3|      T|   4|   2|  health|services|      home|            Home|
|    GP|  F| 16|      U|    GT3|      T|   3|   3|   other|   other|      home|            Home|
|    GP|  M| 16|      U|    LE3|      T|   4|   3|services|   other|reputation|      Reputation|
|    GP|  M| 16|      U|    LE3|      T|   2|   2|   other|   other|      home|            Home|
|    GP|  F| 17|      U|    GT

In [13]:
df['Mjob'].apply(lambda x : str(x).capitalize())

0       At_home
1       At_home
2       At_home
3        Health
4         Other
         ...   
390    Services
391    Services
392       Other
393    Services
394       Other
Name: Mjob, Length: 395, dtype: object

### Step 6. Capitalize both Mjob and Fjob

In [14]:
students_subset.withColumn("col_capitalizada", capitalize_udf(students_subset["Mjob"])).select(F.col('Mjob')).show()
students_subset.withColumn("col_capitalizada", capitalize_udf(students_subset["Fjob"])).select(F.col('Fjob')).show()


+--------+
|    Mjob|
+--------+
| at_home|
| at_home|
| at_home|
|  health|
|   other|
|services|
|   other|
|   other|
|services|
|   other|
| teacher|
|services|
|  health|
| teacher|
|   other|
|  health|
|services|
|   other|
|services|
|  health|
+--------+
only showing top 20 rows

+--------+
|    Fjob|
+--------+
| teacher|
|   other|
|   other|
|services|
|   other|
|   other|
|   other|
| teacher|
|   other|
|   other|
|  health|
|   other|
|services|
|   other|
|   other|
|   other|
|services|
|   other|
|services|
|   other|
+--------+
only showing top 20 rows



In [15]:
students_subset.show()

+------+---+---+-------+-------+-------+----+----+--------+--------+----------+
|school|sex|age|address|famsize|Pstatus|Medu|Fedu|    Mjob|    Fjob|    reason|
+------+---+---+-------+-------+-------+----+----+--------+--------+----------+
|    GP|  F| 18|      U|    GT3|      A|   4|   4| at_home| teacher|    course|
|    GP|  F| 17|      U|    GT3|      T|   1|   1| at_home|   other|    course|
|    GP|  F| 15|      U|    LE3|      T|   1|   1| at_home|   other|     other|
|    GP|  F| 15|      U|    GT3|      T|   4|   2|  health|services|      home|
|    GP|  F| 16|      U|    GT3|      T|   3|   3|   other|   other|      home|
|    GP|  M| 16|      U|    LE3|      T|   4|   3|services|   other|reputation|
|    GP|  M| 16|      U|    LE3|      T|   2|   2|   other|   other|      home|
|    GP|  F| 17|      U|    GT3|      A|   4|   4|   other| teacher|      home|
|    GP|  M| 15|      U|    LE3|      A|   3|   2|services|   other|      home|
|    GP|  M| 15|      U|    GT3|      T|

In [16]:
df['Mjob'].apply(lambda x: str(x).capitalize())
df['Fjob'].apply(lambda x: str(x).capitalize())




0       Teacher
1         Other
2         Other
3      Services
4         Other
         ...   
390    Services
391    Services
392       Other
393       Other
394     At_home
Name: Fjob, Length: 395, dtype: object

### Step 7. Print the last elements of the data set.

In [17]:
spark.createDataFrame(students_subset.tail(10)).show()


+------+---+---+-------+-------+-------+----+----+--------+--------+----------+
|school|sex|age|address|famsize|Pstatus|Medu|Fedu|    Mjob|    Fjob|    reason|
+------+---+---+-------+-------+-------+----+----+--------+--------+----------+
|    MS|  F| 18|      R|    GT3|      T|   2|   2| at_home|   other|     other|
|    MS|  F| 18|      R|    GT3|      T|   4|   4| teacher| at_home|reputation|
|    MS|  F| 19|      R|    GT3|      T|   2|   3|services|   other|    course|
|    MS|  F| 18|      U|    LE3|      T|   3|   1| teacher|services|    course|
|    MS|  F| 18|      U|    GT3|      T|   1|   1|   other|   other|    course|
|    MS|  M| 20|      U|    LE3|      A|   2|   2|services|services|    course|
|    MS|  M| 17|      U|    LE3|      T|   3|   1|services|services|    course|
|    MS|  M| 21|      R|    GT3|      T|   1|   1|   other|   other|    course|
|    MS|  M| 18|      R|    LE3|      T|   3|   2|services|   other|    course|
|    MS|  M| 19|      U|    LE3|      T|

In [18]:
df_subset.tail()


,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,reason,guardian
390,MS,M,20,U,LE3,A,2,2,services,services,course,other
391,MS,M,17,U,LE3,T,3,1,services,services,course,mother
392,MS,M,21,R,GT3,T,1,1,other,other,course,other
393,MS,M,18,R,LE3,T,3,2,services,other,course,mother
394,MS,M,19,U,LE3,T,1,1,other,at_home,course,father


### Step 8. Did you notice the original dataframe is still lowercase? Why is that? Fix it and capitalize Mjob and Fjob.

In [19]:
students_subset = students_subset.withColumn("Mjob", capitalize_udf(students_subset["Mjob"]))
students_subset = students_subset.withColumn("Fjob", capitalize_udf(students_subset["Fjob"]))
students_subset.show()

+------+---+---+-------+-------+-------+----+----+--------+--------+----------+
|school|sex|age|address|famsize|Pstatus|Medu|Fedu|    Mjob|    Fjob|    reason|
+------+---+---+-------+-------+-------+----+----+--------+--------+----------+
|    GP|  F| 18|      U|    GT3|      A|   4|   4| At_home| Teacher|    course|
|    GP|  F| 17|      U|    GT3|      T|   1|   1| At_home|   Other|    course|
|    GP|  F| 15|      U|    LE3|      T|   1|   1| At_home|   Other|     other|
|    GP|  F| 15|      U|    GT3|      T|   4|   2|  Health|Services|      home|
|    GP|  F| 16|      U|    GT3|      T|   3|   3|   Other|   Other|      home|
|    GP|  M| 16|      U|    LE3|      T|   4|   3|Services|   Other|reputation|
|    GP|  M| 16|      U|    LE3|      T|   2|   2|   Other|   Other|      home|
|    GP|  F| 17|      U|    GT3|      A|   4|   4|   Other| Teacher|      home|
|    GP|  M| 15|      U|    LE3|      A|   3|   2|Services|   Other|      home|
|    GP|  M| 15|      U|    GT3|      T|

In [20]:
df_subset['Mjob'] = df_subset['Mjob'].apply(lambda x: str(x).capitalize())
df_subset['Fjob'] = df_subset['Fjob'].apply(lambda x: str(x).capitalize())

In [21]:
df_subset

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,reason,guardian
0,GP,F,18,U,GT3,A,4,4,At_home,Teacher,course,mother
1,GP,F,17,U,GT3,T,1,1,At_home,Other,course,father
2,GP,F,15,U,LE3,T,1,1,At_home,Other,other,mother
3,GP,F,15,U,GT3,T,4,2,Health,Services,home,mother
4,GP,F,16,U,GT3,T,3,3,Other,Other,home,father
...,...,...,...,...,...,...,...,...,...,...,...,...
390,MS,M,20,U,LE3,A,2,2,Services,Services,course,other
391,MS,M,17,U,LE3,T,3,1,Services,Services,course,mother
392,MS,M,21,R,GT3,T,1,1,Other,Other,course,other
393,MS,M,18,R,LE3,T,3,2,Services,Other,course,mother


### Step 9. Create a function called majority that returns a boolean value to a new column called legal_drinker (Consider majority as older than 17 years old)

In [22]:

from pyspark.sql import types as T

def majority(x):
    if x is not None and int(x) > 17:
        return True
    else:
        return False

# Registra la UDF como PySpark Function
majority_udf = F.udf(majority, T.BooleanType())






In [23]:
students_subset = students_subset.withColumn("legal_drinker", majority_udf(F.col("age")))


In [24]:
students_subset.show()


+------+---+---+-------+-------+-------+----+----+--------+--------+----------+-------------+
|school|sex|age|address|famsize|Pstatus|Medu|Fedu|    Mjob|    Fjob|    reason|legal_drinker|
+------+---+---+-------+-------+-------+----+----+--------+--------+----------+-------------+
|    GP|  F| 18|      U|    GT3|      A|   4|   4| At_home| Teacher|    course|         true|
|    GP|  F| 17|      U|    GT3|      T|   1|   1| At_home|   Other|    course|        false|
|    GP|  F| 15|      U|    LE3|      T|   1|   1| At_home|   Other|     other|        false|
|    GP|  F| 15|      U|    GT3|      T|   4|   2|  Health|Services|      home|        false|
|    GP|  F| 16|      U|    GT3|      T|   3|   3|   Other|   Other|      home|        false|
|    GP|  M| 16|      U|    LE3|      T|   4|   3|Services|   Other|reputation|        false|
|    GP|  M| 16|      U|    LE3|      T|   2|   2|   Other|   Other|      home|        false|
|    GP|  F| 17|      U|    GT3|      A|   4|   4|   Other| 

In [25]:
def majority(x):
    if x > 17:
        return True
    else:
        return False
    

df_subset['legal_drinker'] = df_subset['age'].apply(majority)



### Step 10. Multiply every number of the dataset by 10. 
##### I know this makes no sense, don't forget it is just an exercise

In [31]:
students_subset = students_subset \
    .withColumn("age", F.col("age").cast(T.IntegerType())) \
    .withColumn("Medu", F.col("Medu").cast(T.IntegerType())) \
    .withColumn("Fedu", F.col("Fedu").cast(T.IntegerType())) \
    .withColumn("legal_drinker", F.col("legal_drinker").cast(T.BooleanType()))

In [32]:
students_subset.printSchema()

root
 |-- school: string (nullable = true)
 |-- sex: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- address: string (nullable = true)
 |-- famsize: string (nullable = true)
 |-- Pstatus: string (nullable = true)
 |-- Medu: integer (nullable = true)
 |-- Fedu: integer (nullable = true)
 |-- Mjob: string (nullable = true)
 |-- Fjob: string (nullable = true)
 |-- reason: string (nullable = true)
 |-- legal_drinker: boolean (nullable = true)



In [38]:
from pyspark.sql.types import BooleanType, NumericType

numeric_cols = [
    f.name for f in students_subset.schema.fields
    if isinstance(f.dataType, NumericType) and not isinstance(f.dataType, BooleanType)
]

students_scaled = students_subset.select(
    *[
        (F.col(c) * 10).alias(c) if c in numeric_cols else F.col(c)
        for c in students_subset.columns
    ]
)

students_scaled.show()


+------+---+---+-------+-------+-------+----+----+--------+--------+----------+-------------+
|school|sex|age|address|famsize|Pstatus|Medu|Fedu|    Mjob|    Fjob|    reason|legal_drinker|
+------+---+---+-------+-------+-------+----+----+--------+--------+----------+-------------+
|    GP|  F|180|      U|    GT3|      A|  40|  40| At_home| Teacher|    course|         true|
|    GP|  F|170|      U|    GT3|      T|  10|  10| At_home|   Other|    course|        false|
|    GP|  F|150|      U|    LE3|      T|  10|  10| At_home|   Other|     other|        false|
|    GP|  F|150|      U|    GT3|      T|  40|  20|  Health|Services|      home|        false|
|    GP|  F|160|      U|    GT3|      T|  30|  30|   Other|   Other|      home|        false|
|    GP|  M|160|      U|    LE3|      T|  40|  30|Services|   Other|reputation|        false|
|    GP|  M|160|      U|    LE3|      T|  20|  20|   Other|   Other|      home|        false|
|    GP|  F|170|      U|    GT3|      A|  40|  40|   Other| 

In [27]:


df_subset[['age', 'Medu', 'Fedu']] = df_subset[['age', 'Medu', 'Fedu']].astype('int')*10



In [28]:
df_subset

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,reason,guardian,legal_drinker
0,GP,F,180,U,GT3,A,40,40,At_home,Teacher,course,mother,True
1,GP,F,170,U,GT3,T,10,10,At_home,Other,course,father,False
2,GP,F,150,U,LE3,T,10,10,At_home,Other,other,mother,False
3,GP,F,150,U,GT3,T,40,20,Health,Services,home,mother,False
4,GP,F,160,U,GT3,T,30,30,Other,Other,home,father,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
390,MS,M,200,U,LE3,A,20,20,Services,Services,course,other,True
391,MS,M,170,U,LE3,T,30,10,Services,Services,course,mother,False
392,MS,M,210,R,GT3,T,10,10,Other,Other,course,other,True
393,MS,M,180,R,LE3,T,30,20,Services,Other,course,mother,True
